MODEL 2 BUT WITH DURATION_MS

In [2]:
! pip install -q kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
#https://drive.google.com/file/d/1Q1UZbj4qXImimLv91y4wkLKFnJ5YHnuR/view?usp=share_link
from google.colab import drive

# Replace "1234567890abcdefghijk" with the actual file ID
file_id = "1Q1UZbj4qXImimLv91y4wkLKFnJ5YHnuR"

# Get the path to the Kaggle API credentials file
kaggle_json_path = "/content/drive/MyDrive/Project4/kaggle.json"

# Mount your Google Drive
drive.mount('/content/drive')

# Copy the Kaggle API credentials file from Google Drive to the current environment
!cp "/content/drive/MyDrive/Project4/kaggle.json" "/content/kaggle.json"

Mounted at /content/drive


In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download -d amitanshjoshi/spotify-1million-tracks

Dataset URL: https://www.kaggle.com/datasets/amitanshjoshi/spotify-1million-tracks
License(s): ODbL-1.0
 84% 65.0M/77.1M [00:01<00:00, 59.2MB/s]
100% 77.1M/77.1M [00:01<00:00, 62.2MB/s]


In [8]:
! unzip spotify-1million-tracks

Archive:  spotify-1million-tracks.zip
  inflating: spotify_data.csv        


In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import joblib

In [10]:
spotify_data_raw = pd.read_csv('spotify_data.csv')
spotify_data_raw

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.1390,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.5150,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.1450,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.5080,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.2170,171.864,244320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159759,1473391,Nicola Conte,Black Spirits,0m27F0IGHLGAWhqd6ccYst,4,2011,trip-hop,0.373,0.742,10,-6.453,0,0.0736,0.3250,0.000141,0.1590,0.5220,107.951,344013,3
1159760,1473392,Nicola Conte,Quiet Dawn,6er9p611eHEcUCU50j7D57,3,2011,trip-hop,0.516,0.675,7,-7.588,0,0.0326,0.7880,0.000129,0.1300,0.2640,119.897,285067,4
1159761,1473393,Amon Tobin,Morning Ms Candis,7jsMMqxy1tt0rH5FzYcZTQ,2,2011,trip-hop,0.491,0.440,5,-8.512,1,0.0274,0.4770,0.003130,0.0936,0.0351,100.076,214253,4
1159762,1473394,Peace Orchestra,Happy Christmas (War Is Over),77lA1InUaXztuRk2vOzD1S,0,2011,trip-hop,0.480,0.405,0,-13.343,1,0.0276,0.4310,0.000063,0.1250,0.2020,133.885,239133,3


In [11]:
apop1 = pd.read_csv('artist_pop_1.csv')
apop2 = pd.read_csv('artist_pop_2.csv')
apop3 = pd.read_csv('artist_pop_3.csv')
apop4 = pd.read_csv('artist_pop_4.csv')
artist_pop = pd.concat([apop1, apop2, apop3, apop4])
artist_pop['artist_popularity'] = artist_pop['popularity']
artist_pop = artist_pop[['artist_name', 'artist_popularity']]
artist_pop

,artist_name,artist_popularity
0,Jason Mraz,69
1,Joshua Hyslop,46
2,Boyce Avenue,66
3,Andrew Belle,51
4,Chris Smither,40
...,...,...
19061,Dabruck & Klein,11
19062,Andrea Bertolini,11
19063,PureNRG,14
19064,The Slow Taste Of Night Air,0


In [12]:
spotify_data = spotify_data_raw.merge(artist_pop, on='artist_name')
spotify_data = spotify_data.dropna()
spotify_data = spotify_data.drop_duplicates()
spotify_data

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_popularity
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,...,1,0.0429,0.69400,0.000000,0.1150,0.139,133.406,240166,3,69
3,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,...,1,0.0258,0.47700,0.000014,0.0974,0.515,140.182,216387,4,69
6,8,Jason Mraz,Living in the Moment,3ce7k1L4EkZppZPz1EJWTS,44,2012,acoustic,0.650,0.628,7,...,1,0.0232,0.04830,0.000000,0.1190,0.700,84.141,235080,4,69
9,13,Jason Mraz,The Woman I Love,0AYG9AcwqEeqgAw3OvshPC,40,2012,acoustic,0.591,0.647,4,...,1,0.0277,0.06700,0.000066,0.2310,0.678,79.680,190752,4,69
12,21,Jason Mraz,Winter Wonderland,0zXjeGMMe7esPTvqJCN5d8,36,2012,acoustic,0.609,0.359,5,...,1,0.0468,0.77700,0.000000,0.2210,0.645,145.178,127080,4,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204578,1472581,Andrea Bertolini,I Need You - Instrumental Mix,4DyoeURyOkgwdIkw1rDDq3,11,2011,trance,0.754,0.723,7,...,1,0.1040,0.07370,0.904000,0.1030,0.134,128.000,485156,4,11
1204579,1472583,Andrea Bertolini,I Need You - Invisible Brothers Remix,2ukiz06Z3V0Cbbd366St4z,11,2011,trance,0.772,0.769,4,...,0,0.1220,0.00148,0.493000,0.0873,0.756,129.997,472573,4,11
1204580,1472586,Andrea Bertolini,I Need You - Kobana Remix,2ZZXt7c1flVyNCkqirsr8G,11,2011,trance,0.771,0.750,2,...,1,0.0845,0.08790,0.394000,0.0739,0.410,126.996,381527,4,11
1204581,1472589,Andrea Bertolini,I Need You - Vocal Mix,0N9r3418GBS1a6UlL3DZtP,11,2011,trance,0.716,0.715,4,...,0,0.0786,0.06370,0.623000,0.1000,0.190,128.015,485156,4,11


In [13]:
spotify_data_binned = spotify_data.drop(columns=['artist_name', 'track_name', 'year', 'track_id', 'Unnamed: 0']) #, 'duration_ms'
spotify_data_binned

,popularity,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_popularity
0,68,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.69400,0.000000,0.1150,0.139,133.406,240166,3,69
3,50,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.47700,0.000014,0.0974,0.515,140.182,216387,4,69
6,44,acoustic,0.650,0.628,7,-7.160,1,0.0232,0.04830,0.000000,0.1190,0.700,84.141,235080,4,69
9,40,acoustic,0.591,0.647,4,-8.340,1,0.0277,0.06700,0.000066,0.2310,0.678,79.680,190752,4,69
12,36,acoustic,0.609,0.359,5,-6.715,1,0.0468,0.77700,0.000000,0.2210,0.645,145.178,127080,4,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204578,11,trance,0.754,0.723,7,-8.360,1,0.1040,0.07370,0.904000,0.1030,0.134,128.000,485156,4,11
1204579,11,trance,0.772,0.769,4,-4.436,0,0.1220,0.00148,0.493000,0.0873,0.756,129.997,472573,4,11
1204580,11,trance,0.771,0.750,2,-7.475,1,0.0845,0.08790,0.394000,0.0739,0.410,126.996,381527,4,11
1204581,11,trance,0.716,0.715,4,-8.004,0,0.0786,0.06370,0.623000,0.1000,0.190,128.015,485156,4,11


In [14]:
def pop_binning_function(x):
    if x <= 25:
        return 0
    elif 25 < x <= 50:
        return 1
    elif 50 < x <= 75:
        return 2
    else:
        return 3

spotify_data_binned['popularity'] = spotify_data_binned['popularity'].apply(pop_binning_function)
spotify_data_binned

,popularity,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_popularity
0,2,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.69400,0.000000,0.1150,0.139,133.406,240166,3,69
3,1,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.47700,0.000014,0.0974,0.515,140.182,216387,4,69
6,1,acoustic,0.650,0.628,7,-7.160,1,0.0232,0.04830,0.000000,0.1190,0.700,84.141,235080,4,69
9,1,acoustic,0.591,0.647,4,-8.340,1,0.0277,0.06700,0.000066,0.2310,0.678,79.680,190752,4,69
12,1,acoustic,0.609,0.359,5,-6.715,1,0.0468,0.77700,0.000000,0.2210,0.645,145.178,127080,4,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204578,0,trance,0.754,0.723,7,-8.360,1,0.1040,0.07370,0.904000,0.1030,0.134,128.000,485156,4,11
1204579,0,trance,0.772,0.769,4,-4.436,0,0.1220,0.00148,0.493000,0.0873,0.756,129.997,472573,4,11
1204580,0,trance,0.771,0.750,2,-7.475,1,0.0845,0.08790,0.394000,0.0739,0.410,126.996,381527,4,11
1204581,0,trance,0.716,0.715,4,-8.004,0,0.0786,0.06370,0.623000,0.1000,0.190,128.015,485156,4,11


In [15]:
spotify_data_binned = pd.get_dummies(spotify_data_binned, columns=['genre'])
spotify_data_binned

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,genre_ska,genre_sleep,genre_songwriter,genre_soul,genre_spanish,genre_swedish,genre_tango,genre_techno,genre_trance,genre_trip-hop
0,2,0.483,0.303,4,-10.058,1,0.0429,0.69400,0.000000,0.1150,...,False,False,False,False,False,False,False,False,False,False
3,1,0.572,0.454,3,-10.286,1,0.0258,0.47700,0.000014,0.0974,...,False,False,False,False,False,False,False,False,False,False
6,1,0.650,0.628,7,-7.160,1,0.0232,0.04830,0.000000,0.1190,...,False,False,False,False,False,False,False,False,False,False
9,1,0.591,0.647,4,-8.340,1,0.0277,0.06700,0.000066,0.2310,...,False,False,False,False,False,False,False,False,False,False
12,1,0.609,0.359,5,-6.715,1,0.0468,0.77700,0.000000,0.2210,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204578,0,0.754,0.723,7,-8.360,1,0.1040,0.07370,0.904000,0.1030,...,False,False,False,False,False,False,False,False,True,False
1204579,0,0.772,0.769,4,-4.436,0,0.1220,0.00148,0.493000,0.0873,...,False,False,False,False,False,False,False,False,True,False
1204580,0,0.771,0.750,2,-7.475,1,0.0845,0.08790,0.394000,0.0739,...,False,False,False,False,False,False,False,False,True,False
1204581,0,0.716,0.715,4,-8.004,0,0.0786,0.06370,0.623000,0.1000,...,False,False,False,False,False,False,False,False,True,False


In [16]:
# Split our preprocessed data into our features and target arrays
y = spotify_data_binned.popularity.values
X = spotify_data_binned.drop(columns="popularity").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)
y_train = tf.one_hot(y_train, depth=4)
y_test = tf.one_hot(y_test, depth=4)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=4, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                7760      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 4)                 324       
                                                                 
Total params: 21044 (82.20 KB)
Trainable params: 21044 (82.20 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model

nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Epoch 1/50
25075/25075 [==============================] - 77s 3ms/step - loss: 0.5127 - accuracy: 0.7720 - val_loss: 0.5024 - val_accuracy: 0.7765
Epoch 2/50
25075/25075 [==============================] - 87s 3ms/step - loss: 0.4994 - accuracy: 0.7777 - val_loss: 0.4969 - val_accuracy: 0.7787
Epoch 3/50
25075/25075 [==============================] - 80s 3ms/step - loss: 0.4952 - accuracy: 0.7801 - val_loss: 0.4973 - val_accuracy: 0.7797
Epoch 4/50
25075/25075 [==============================] - 75s 3ms/step - loss: 0.4928 - accuracy: 0.7815 - val_loss: 0.4941 - val_accuracy: 0.7813
Epoch 5/50
25075/25075 [==============================] - 76s 3ms/step - loss: 0.4910 - accuracy: 0.7824 - val_loss: 0.4915 - val_accuracy: 0.7824
Epoch 6/50
25075/25075 [==============================] - 92s 4ms/step - loss: 0.4895 - accuracy: 0.7832 - val_loss: 0.4922 - val_accuracy: 0.7819
Epoch 7/50
25075/25075 [==============================] - 74s 3ms/step - loss: 0.4884 - accuracy: 0.7834 - val_loss: 0

In [23]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8359/8359 - 24s - loss: 0.4923 - accuracy: 0.7842 - 24s/epoch - 3ms/step
Loss: 0.4922727644443512, Accuracy: 0.7842446565628052


In [22]:
nn.save("./SpotifyModelOptimized_5.h5")

from google.colab import files
files.download('./SpotifyModelOptimized_5.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>